## **Análise de Demonstrações Financeiras - Embraer S.A. (2020 a 2024)**

In [56]:
# bibliotecas importadas
import requests
import zipfile
import os
import pandas as pd
import io
import numpy as np
from openpyxl import Workbook

### Baixar as DFPs (2021–2025) da Embraer em XBRL diretamente do site da CVM

In [57]:
# ===== Configurações =====
ANOS = range(2021, 2025)  # 2020, 2021, 2022, 2023, 2024
URL_BASE = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_{}.zip"
EMPRESA = "EMBRAER S.A."
PASTA_ZIP = "zips"
PASTA_SAIDA = "saida"

# Nome-padrão dentro do zip para os demonstrativos consolidados
DEMONSTRATIVOS = {
    "Ativo": "BPA_con",
    "Passivo": "BPP_con",
    "DRE": "DRE_con",
    "DFC": "DFC_MI_con"
}

os.makedirs(PASTA_ZIP, exist_ok=True)
os.makedirs(PASTA_SAIDA, exist_ok=True)

dfs = {nome: [] for nome in DEMONSTRATIVOS.keys()}

for ano in ANOS:
    print(f"🔽 Baixando ano {ano}...")
    url = URL_BASE.format(ano)
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"❌ Não conseguiu baixar para o ano {ano}. Status: {resp.status_code}")
        continue

    # Salvar zip localmente opcionalmente
    caminho_zip = os.path.join(PASTA_ZIP, f"dfp_cia_aberta_{ano}.zip")
    with open(caminho_zip, "wb") as f:
        f.write(resp.content)
    print(f"✅ Zip salvo: {caminho_zip}")

    # Abrir zip em memória
    with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
        nomes_zip = z.namelist()
        for nome_dem, padrao in DEMONSTRATIVOS.items():
            # encontrar arquivo CSV consolidado que contenha o padrão
            # ex: “dfp_cia_aberta_BPA_con_2023.csv”
            possiveis = [f for f in nomes_zip if padrao in f and f.endswith(".csv")]
            if not possiveis:
                print(f"⚠️ Não achei arquivo {padrao} para ano {ano}")
                continue

            nome_arq = possiveis[0]
            print(f"   ↳ Extraindo {nome_arq} para {nome_dem}, ano {ano}")

            with z.open(nome_arq) as arq:
                # Ler CSV com encoding e separador certos; pode ajustar dependendo do CSV
                df = pd.read_csv(arq, sep=";", encoding="latin1", dtype=str)

            # Filtrar para Embraer
            df_emb = df[df["DENOM_CIA"].str.strip().str.upper() == EMPRESA.upper()]
            if df_emb.empty:
                print(f"      ‼️ Nenhum registro para Embraer no {nome_dem}, ano {ano}")
            else:
                df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
                dfs[nome_dem].append(df_emb)

for nome_dem, lista in dfs.items():
    if not lista:
        print(f"⚠️ Sem dados para {nome_dem} nos anos especificados.")
        continue
    df_total = pd.concat(lista, ignore_index=True)
    caminho_saida = os.path.join(PASTA_SAIDA, f"{nome_dem}_Embraer_2020-2024.csv")
    df_total.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=";")
    print(f"✅ Salvou {caminho_saida}")


🔽 Baixando ano 2021...
✅ Zip salvo: zips\dfp_cia_aberta_2021.zip
   ↳ Extraindo dfp_cia_aberta_BPA_con_2021.csv para Ativo, ano 2021


C:\Users\Madu\AppData\Local\Temp\ipykernel_16956\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year


   ↳ Extraindo dfp_cia_aberta_BPP_con_2021.csv para Passivo, ano 2021


C:\Users\Madu\AppData\Local\Temp\ipykernel_16956\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_16956\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year


   ↳ Extraindo dfp_cia_aberta_DRE_con_2021.csv para DRE, ano 2021
   ↳ Extraindo dfp_cia_aberta_DFC_MI_con_2021.csv para DFC, ano 2021


...

   ↳ Extraindo dfp_cia_aberta_BPP_con_2024.csv para Passivo, ano 2024


C:\Users\Madu\AppData\Local\Temp\ipykernel_16956\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year
C:\Users\Madu\AppData\Local\Temp\ipykernel_16956\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year


   ↳ Extraindo dfp_cia_aberta_DRE_con_2024.csv para DRE, ano 2024
   ↳ Extraindo dfp_cia_aberta_DFC_MI_con_2024.csv para DFC, ano 2024


C:\Users\Madu\AppData\Local\Temp\ipykernel_16956\355045126.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = pd.to_datetime(df_emb["DT_FIM_EXERC"], format="%Y-%m-%d").dt.year


✅ Salvou saida\Ativo_Embraer_2020-2024.csv
✅ Salvou saida\Passivo_Embraer_2020-2024.csv
✅ Salvou saida\DRE_Embraer_2020-2024.csv
✅ Salvou saida\DFC_Embraer_2020-2024.csv


In [59]:
for nome_dem in DEMONSTRATIVOS.keys():
    caminho = os.path.join(PASTA_SAIDA, f"{nome_dem}_Embraer_2020-2024.csv")
    if not os.path.exists(caminho):
        continue
    df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig", dtype=str)
    # Selecionar colunas relevantes (conferir futuramente)
    df_sel = df[["CD_CONTA", "DS_CONTA", "ANO", "VL_CONTA"]]
    # Pivotar para ter anos como colunas
    df_pivot = df_sel.pivot_table(index=["CD_CONTA", "DS_CONTA"], 
                                  columns="ANO", values="VL_CONTA", aggfunc="first").reset_index()
    
    novo = os.path.join(PASTA_SAIDA, f"{nome_dem}_contabil_2020-2024.csv")
    df_pivot.to_csv(novo, index=False, encoding="utf-8-sig", sep=";")
    print(f"📦 Salvou versão contabil: {novo}")

📦 Salvou versão contabil: saida\Ativo_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\Passivo_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\DRE_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\DFC_contabil_2020-2024.csv


In [60]:
ARQUIVO_XLSM = "Demonstrações_Embraer.xlsm"

arquivos = {
    "Ativo": "Ativo_Embraer_2020-2024.csv",
    "Passivo": "Passivo_Embraer_2020-2024.csv",
    "DRE": "DRE_Embraer_2020-2024.csv",
    "DFC": "DFC_Embraer_2020-2024.csv"
}

with pd.ExcelWriter(ARQUIVO_XLSM, engine="openpyxl") as writer:
    for aba, nome_arq in arquivos.items():
        caminho = os.path.join(PASTA_SAIDA, nome_arq)
        if not os.path.exists(caminho):
            print(f"⚠️ Arquivo não encontrado: {caminho}")
            continue

        df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")

        # Pivotar: Conta x Ano
        if "CD_CONTA" in df.columns and "DS_CONTA" in df.columns:
            df["Conta"] = df["CD_CONTA"] + " - " + df["DS_CONTA"]
        else:
            df["Conta"] = df["DS_CONTA"]

        # Converter valores para numérico (quando possível)
        df["VALOR"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")

        tabela = df.pivot_table(
            index="Conta",
            columns="ANO",
            values="VALOR",
            aggfunc="first"
        ).reset_index()

        tabela.to_excel(writer, sheet_name=aba, index=False)

print(f"✅ Arquivo Excel criado: {ARQUIVO_XLSM}")

✅ Arquivo Excel criado: Demonstrações_Embraer.xlsm


### Correção monetária pelo IPCA para 2024
O Balanço é uma foto da posição financeira da empresa em uma data específica. Ele representa saldos acumulados ou estoques de valor.

In [61]:
# IPCA anual acumulado (coletado no https://www.yahii.com.br/ipca.html)
ipca = {
    2020: 0.0452,
    2021: 0.1006,
    2022: 0.0579,
    2023: 0.0462,
    2024: 0.0462
}

# Calcular fatores de correção para base 2024
fator_2024 = 1.0
fator_2023 = (1 + ipca[2024])
fator_2022 = (1 + ipca[2023]) * fator_2023
fator_2021 = (1 + ipca[2022]) * fator_2022
fator_2020 = (1 + ipca[2021]) * fator_2021

fatores_balanco = {
    2020: fator_2020,
    2021: fator_2021,
    2022: fator_2022,
    2023: fator_2023,
    2024: fator_2024
}

print("\nFatores de Correção para base 2024 - Balanço (Ativo/Passivo):")
for ano, fator in fatores_balanco.items():
    print(f"  - Ano {ano}: {fator:.4f}")



Fatores de Correção para base 2024 - Balanço (Ativo/Passivo):
  - Ano 2020: 1.2744
  - Ano 2021: 1.1579
  - Ano 2022: 1.0945
  - Ano 2023: 1.0462
  - Ano 2024: 1.0000


#### Taxa média geométrica para inflação iquida e índices do DRE e do DFC (2020-2024)
A receita de janeiro foi gerada com um poder de compra diferente da de dezembro, então é preciso corrigir.
Para corrigir isso, é preciso de uma taxa que represente a inflação média que incidiu sobre os fluxos ao longo do ano. A média geométrica é a forma matematicamente correta de encontrar essa taxa média para juros compostos (que é o caso da inflação).

In [62]:
# Taxa média geométrica para inflação iquida e índices
# IPCA mensal (coletado no https://www.yahii.com.br/ipca.html)
ipca_mensal = {
    2020: [0.0021, 0.0025, 0.0007, -0.0031, -0.0038, 0.0026, 0.0036, 0.0024, 0.0064, 0.0086, 0.0089, 0.0135],
    2021: [0.0025, 0.0086, 0.0093, 0.0031, 0.0083, 0.0053, 0.0096, 0.0087, 0.0116, 0.0125, 0.0095, 0.0073],
    2022: [0.0054, 0.0101, 0.0162, 0.0106, 0.0047, 0.0067, -0.0068, -0.0036, -0.0029, 0.0059, 0.0041, 0.0062],
    2023: [0.0053, 0.0084, 0.0071, 0.0061, 0.0023, -0.0008, 0.0012, 0.0023, 0.0026, 0.0024, 0.0028, 0.0056]
}

print("--- Calculando Taxas Médias Geométricas Mensais ---")
taxas_medias_geometricas = {}

for ano, taxas_mensais in ipca_mensal.items():
    # A fórmula é: (produto de (1 + i_mês))^(1/12) - 1
    produto_acumulado = np.prod([1 + taxa for taxa in taxas_mensais])
    taxa_media = (produto_acumulado ** (1/12)) - 1
    taxas_medias_geometricas[ano] = taxa_media
    print(f"  - Taxa Média Geométrica de {ano}: {taxa_media:.6f} (ou {taxa_media*100:.4f}%)")

# Fatores para DRE e DFC - Baseados na taxa média geométrica + inflação cheia dos anos seguintes
fatores_fluxo = {
    2024: 1.0, # Ano base, não tem correção
    2023: (1 + taxas_medias_geometricas[2023]) * (1 + ipca[2024]),
    2022: (1 + taxas_medias_geometricas[2022]) * fator_2023,
    2021: (1 + taxas_medias_geometricas[2021]) * fator_2022,
    2020: (1 + taxas_medias_geometricas[2020]) * fator_2021
}

print("\nFatores para DRE e DFC (Fluxos):")
for ano, fator in sorted(fatores_fluxo.items()):
    print(f"  - Ano {ano}: {fator:.4f}")


--- Calculando Taxas Médias Geométricas Mensais ---
  - Taxa Média Geométrica de 2020: 0.003689 (ou 0.3689%)
  - Taxa Média Geométrica de 2021: 0.008021 (ou 0.8021%)
  - Taxa Média Geométrica de 2022: 0.004697 (ou 0.4697%)
  - Taxa Média Geométrica de 2023: 0.003772 (ou 0.3772%)

Fatores para DRE e DFC (Fluxos):
  - Ano 2020: 1.1622
  - Ano 2021: 1.1033
  - Ano 2022: 1.0511
  - Ano 2023: 1.0501
  - Ano 2024: 1.0000


In [ ]:
'''
# Aplicar a correção a cada arquivo na pasta "saida"
for nome_arq in os.listdir(PASTA_SAIDA):
    if not nome_arq.lower().endswith('.csv'):
        continue
    
    caminho = os.path.join(PASTA_SAIDA, nome_arq)
    df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")
    nome_upper = nome_arq.upper()

    # Escolher o conjunto de fatores
    if "ATIVO" in nome_upper or "PASSIVO" in nome_upper:
        fatores_usados = fatores_balanco
        tipo = "Balanço"
    elif "DRE" in nome_upper or "DFC" in nome_upper:
        fatores_usados = fatores_fluxo
        tipo = "Fluxo"
    else:
        print(f"⚠️  Arquivo ignorado (tipo não identificado): {nome_arq}")
        continue

    if "ANO" in df.columns and "VL_CONTA" in df.columns:
        df["VALOR_CORRIGIDO_2024"] = df.apply( lambda x: pd.to_numeric(x["VL_CONTA"], errors="coerce") * fatores_usados.get(int(x["ANO"]), 1), axis=1 )
            
# Salvar com os valores corrigidos
nome_corrigido = nome_arq.replace(".csv", "_corrigido_2024.csv")
df.to_csv(os.path.join(PASTA_SAIDA, nome_corrigido), sep=";", encoding="utf-8-sig", index=False)
print(f"✅ Corrigido e salvo: {nome_corrigido}")

'''

'\n# Aplicar a correção a cada arquivo na pasta "saida"\nfor nome_arq in os.listdir(PASTA_SAIDA):\n    if not nome_arq.lower().endswith(\'.csv\'):\n        continue\n\n    caminho = os.path.join(PASTA_SAIDA, nome_arq)\n    df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")\n    nome_upper = nome_arq.upper()\n\n    # Escolher o conjunto de fatores\n    if "ATIVO" in nome_upper or "PASSIVO" in nome_upper:\n        fatores_usados = fatores_balanco\n        tipo = "Balanço"\n    elif "DRE" in nome_upper or "DFC" in nome_upper:\n        fatores_usados = fatores_fluxo\n        tipo = "Fluxo"\n    else:\n        print(f"⚠️  Arquivo ignorado (tipo não identificado): {nome_arq}")\n        continue\n\n    if "ANO" in df.columns and "VL_CONTA" in df.columns:\n        df["VALOR_CORRIGIDO_2024"] = df.apply( lambda x: pd.to_numeric(x["VL_CONTA"], errors="coerce") * fatores_usados.get(int(x["ANO"]), 1), axis=1 )\n\n# Salvar com os valores corrigidos\nnome_corrigido = nome_arq.replace(".csv", "_

In [63]:
# Aplicar a correção a cada arquivo na pasta "saida"

for nome_dem in DEMONSTRATIVOS.keys():
    caminho_entrada = os.path.join(PASTA_SAIDA, f"{nome_dem}_Embraer_2020-2024.csv")
    
    if not os.path.exists(caminho_entrada):
        print(f"AVISO: Arquivo de entrada não encontrado, pulando: {caminho_entrada}")
        continue
    
    print(f"-> Processando: {caminho_entrada}")
    df = pd.read_csv(caminho_entrada, sep=";", encoding="utf-8-sig")
    
    # --- 1. ESCOLHER O FATOR DE CORREÇÃO ---
    if nome_dem in ["Ativo", "Passivo"]:
        fatores_usados = fatores_balanco
        print("   Tipo: Balanço. Usando fatores de balanço.")
    else: # DRE e DFC
        fatores_usados = fatores_fluxo
        print("   Tipo: Fluxo. Usando fatores de fluxo.")
        
    # --- 2. APLICAR A CORREÇÃO (ANTES DE PIVOTAR) ---
    df['VL_CONTA'] = pd.to_numeric(df['VL_CONTA'], errors='coerce')
    df['ANO'] = pd.to_numeric(df['ANO'], errors='coerce').fillna(0).astype(int)
    
    df['VALOR_AJUSTADO'] = df.apply(
        lambda linha: linha['VL_CONTA'] * fatores_usados.get(linha['ANO'], 1.0),
        axis=1
    )
    print("   Correção pela inflação aplicada.")
    
    nome_corrigido = f"{nome_dem}_Embraer_2020-2024_corrigido_2024.csv"
    caminho_saida = os.path.join(PASTA_SAIDA, nome_corrigido)
    
    df.to_csv(caminho_saida, sep=";", encoding="utf-8-sig", index=False)
    print(f"✅ Salvo com sucesso como: {nome_corrigido}\n")
    
print("--- Processo finalizado. ---")

-> Processando: saida\Ativo_Embraer_2020-2024.csv
   Tipo: Balanço. Usando fatores de balanço.
   Correção pela inflação aplicada.
✅ Salvo com sucesso como: Ativo_Embraer_2020-2024_corrigido_2024.csv

-> Processando: saida\Passivo_Embraer_2020-2024.csv
   Tipo: Balanço. Usando fatores de balanço.
   Correção pela inflação aplicada.
✅ Salvo com sucesso como: Passivo_Embraer_2020-2024_corrigido_2024.csv

-> Processando: saida\DRE_Embraer_2020-2024.csv
   Tipo: Fluxo. Usando fatores de fluxo.
   Correção pela inflação aplicada.
✅ Salvo com sucesso como: DRE_Embraer_2020-2024_corrigido_2024.csv

-> Processando: saida\DFC_Embraer_2020-2024.csv
   Tipo: Fluxo. Usando fatores de fluxo.
   Correção pela inflação aplicada.
✅ Salvo com sucesso como: DFC_Embraer_2020-2024_corrigido_2024.csv

--- Processo finalizado. ---


### Após correção com o IPCA de 2024


In [66]:
ARQUIVO_XLSM = "Demonstrações_Embraer_Ajustado.xlsm"

arquivos_corrigidos = {
    "Ativo": "Ativo_Embraer_2020-2024_corrigido_2024.csv",
    "Passivo": "Passivo_Embraer_2020-2024_corrigido_2024.csv",
    "DRE": "DRE_Embraer_2020-2024_corrigido_2024.csv",
    "DFC": "DFC_Embraer_2020-2024_corrigido_2024.csv"
}

with pd.ExcelWriter(ARQUIVO_XLSM, engine="openpyxl") as writer:
    for aba, nome_arq in arquivos_corrigidos.items():
        caminho = os.path.join(PASTA_SAIDA, nome_arq)
        if not os.path.exists(caminho):
            print(f"⚠️ Arquivo não encontrado: {caminho}")
            continue

        df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")

        # Converte valores corrigidos para numérico
        df["VALOR_AJUSTADO"] = pd.to_numeric(df["VALOR_AJUSTADO"], errors="coerce")

        # Pivotar: Conta x Ano (com valores já corrigidos)
        tabela_corrigida = df.pivot_table(
            index=["CD_CONTA", "DS_CONTA"],
            columns="ANO",
            values="VALOR_AJUSTADO",
            aggfunc="first"
        ).reset_index()

        # Exporta cada aba para o Excel
        tabela_corrigida = tabela_corrigida.sort_values(by="CD_CONTA")
        tabela_corrigida.to_excel(writer, sheet_name=aba, index=False)

print(f"Arquivo Excel final criado com sucesso: {ARQUIVO_XLSM}")
print("Todas as guias foram corrigidas e consolidadas com base no IPCA de 2024.")


Arquivo Excel final criado com sucesso: Demonstrações_Embraer_Ajustado.xlsm
Todas as guias foram corrigidas e consolidadas com base no IPCA de 2024.
